### Fråga 1

import pandas as pd  
import tkinter as tk  
from tkinter import ttk, filedialog, messagebox  

# Läs in CSV-filen
df = pd.read_csv('test_data.csv')

class VehicleViewerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Autostrada - CSV Viewer")
        self.filtered_df = df.copy()  # Save filtered dataframe here

        # Set window icon
        self.set_window_icon()

        # Style configurations
        style = ttk.Style()
        style.configure("Treeview", rowheight=30, font=('Arial', 10))
        style.configure("Treeview.Heading", font=('Arial', 12, 'bold'))
        style.configure("Treeview", background="#F5F5F5", foreground="#000000", fieldbackground="#FFFFFF")
        style.configure('TButton', font=('Arial', 10), padding=6)

        # Create a main frame
        main_frame = tk.Frame(root, bg="#EAEAEA")
        main_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)

        # Create filter frame
        filter_frame = tk.Frame(main_frame, bg="#EAEAEA")
        filter_frame.grid(row=0, column=0, sticky="ew")

        # Add a label and entry for filtering
        self.filter_label = tk.Label(filter_frame, text="Filter (Column:Value):", bg="#EAEAEA")
        self.filter_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")

        self.filter_entry = tk.Entry(filter_frame, width=40)
        self.filter_entry.grid(row=0, column=1, padx=5, pady=5)

        self.filter_button = tk.Button(filter_frame, text="Apply Filter", command=self.apply_filter)
        self.filter_button.grid(row=0, column=2, padx=5, pady=5)

        # Result Display as a table
        self.tree = ttk.Treeview(main_frame, columns=[col for col in df.columns], show='headings')
        self.tree.grid(row=1, column=0, padx=10, pady=10, sticky='nsew')

        # Scrollbars
        self.vsb = ttk.Scrollbar(main_frame, orient="vertical", command=self.tree.yview)
        self.vsb.grid(row=1, column=1, sticky='ns')
        self.tree.configure(yscrollcommand=self.vsb.set)

        self.hsb = ttk.Scrollbar(main_frame, orient="horizontal", command=self.tree.xview)
        self.hsb.grid(row=2, column=0, columnspan=2, sticky='ew')
        self.tree.configure(xscrollcommand=self.hsb.set)

        # Set column headings and initial column width
        for col in df.columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100, stretch=tk.NO)  # Initial width, stretch turned off

        self.auto_adjust_column_width()

        # Row count label
        self.row_count_label = tk.Label(main_frame, text=f"Antal rader: {len(df)}", font=('Arial', 10, 'bold'), bg="#EAEAEA")
        self.row_count_label.grid(row=2, column=0, pady=10, sticky="w")

        # Configure grid weights
        main_frame.grid_rowconfigure(1, weight=1)
        main_frame.grid_columnconfigure(0, weight=1)

        # Make the main window resize with the content
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        # Display the full data
        self.display_results(self.filtered_df)

    def set_window_icon(self):
        try:
            # Load and set the window icon
            self.icon_image = tk.PhotoImage(file='icon.png')  # Path to your icon file
            self.root.iconphoto(True, self.icon_image)
        except tk.TclError:
            # Handle error if icon cannot be loaded
            print("Error loading icon image. Make sure the file path is correct and the file is a valid image format.")

    def auto_adjust_column_width(self):
        # Create a label for text measurement
        label = tk.Label(self.root, font=('Arial', 10))
        max_width = {col: len(self.tree.heading(col, 'text')) * 10 for col in self.tree["columns"]}  # Start with heading width

        # Check all column values
        for item in self.tree.get_children():
            row_values = self.tree.item(item, 'values')
            for col, value in zip(self.tree["columns"], row_values):
                label.config(text=value)
                text_width = label.winfo_reqwidth()
                max_width[col] = max(max_width[col], text_width)

        # Set column width based on the widest content
        for col, width in max_width.items():
            self.tree.column(col, width=width + 20)  # Add extra padding

    def display_results(self, df_to_display):
        # Clear previous results
        for item in self.tree.get_children():
            self.tree.delete(item)

        # Display results
        for _, row in df_to_display.iterrows():
            self.tree.insert("", "end", values=row.tolist())

    def apply_filter(self):
        filter_text = self.filter_entry.get()
        if not filter_text:
            self.filtered_df = df.copy()  # If filter is empty, reset to full DataFrame
        else:
            try:
                column, value = filter_text.split(':', 1)
                column = column.strip()
                value = value.strip()
                if column in df.columns:
                    self.filtered_df = df[df[column].astype(str).str.contains(value, case=False, na=False)]
                else:
                    messagebox.showwarning("Invalid Column", f"Column '{column}' does not exist.")
                    self.filtered_df = df.copy()
            except ValueError:
                messagebox.showwarning("Invalid Filter", "Filter format should be 'Column:Value'.")
                self.filtered_df = df.copy()

        # Update the display with filtered data
        self.display_results(self.filtered_df)
        self.row_count_label.config(text=f"Antal rader: {len(self.filtered_df)}")

    def export_to_csv(self):
        # Ask user for a file name and location
        file_path = filedialog.asksaveasfilename(defaultextension=".csv",
                                                 filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
        if not file_path:
            return  # User canceled the save dialog

        try:
            # Export DataFrame to CSV with UTF-8 encoding
            self.filtered_df.to_csv(file_path, index=False, encoding='utf-8-sig')
            messagebox.showinfo("Export Success", "Data exported successfully!")
        except Exception as e:
            messagebox.showerror("Export Error", f"An error occurred while exporting the data:\n{e}")


# Create the application window
root = tk.Tk()
app = VehicleViewerApp(root)
root.mainloop()

### Förklaring

Koden skapar ett graphical user interface (GUI) med hjälp av Tkinter för att visa data från en CSV-fil med följande steg:  
1.CSV-filen 'test_data.csv' läses in i en pandas dataframe.  
2. Klassen 'VehicleViewerApp' är definierad för att skapa huvudapplikationen.  
3.Tkinter används för att skapa ett fönster som har en tabellvisning med hjälp av Treeview och inmatningsfält för att filtrera data.  
4.Användare kan ange ett filter för att komma till den data de vill ha.  
5. Data i CSV-filen visas i en tabell som har scrollbars.  
6. Filtrerad data kan exporteras till en csv-fil.  

### Fråga 2

### 6 -Klass Uppgifter

1.

In [1]:
a = 10
b = [5, 7, 3]
c = {2, 7, 1, 8, 2, 8}

def my_fun (x) :
    return 2*2

d = my_fun

In [2]:
print(type(a))
print(type(b))
print(type(c))
print(type(d))

<class 'int'>
<class 'list'>
<class 'set'>
<class 'function'>


2. (a)

In [3]:
my_variable = (1, 1, 2, 3, 5)

In [4]:
print(isinstance(my_variable, tuple))

True


2. (b)

In [5]:
print(isinstance(my_variable, list))

False


3.

In [6]:
class FruitProduct:
    """A class representing fruit products in a grocery store."""

    def __init__(self, price, quantity):
        self.price= price
        self.quantity= quantity

In [7]:
swedish_apples = FruitProduct(52, 1)

In [8]:
swedish_apples.__doc__

'A class representing fruit products in a grocery store.'

In [9]:
print("Price:", swedish_apples.price)
print("Quantity:", swedish_apples.quantity)

Price: 52
Quantity: 1


In [10]:
type(swedish_apples)

__main__.FruitProduct

In [11]:
isinstance(swedish_apples, FruitProduct)

True

4.

In [12]:
class Squares():
    
    def __init__(self, side_length):
        self.side_length= side_length

    def perimeter(self):
        return self.side_length * 4

    def area(self):
        return self.side_length ** 2



In [13]:
my_square= Squares(8)

In [14]:
print('Perimeter:',my_square.perimeter())
print('Area:',my_square.area())

Perimeter: 32
Area: 64


5. (a)

In [15]:
class DescriptiveStatistics():
    """This class provides functionality for calculating descriptive statistics from a list."""

    def __init__(self):
        self.data=[] #Initialise class with empty list to store data

    def add_data(self, data): 
        if isinstance(data, list):
            self.data.extend(data) #Adds a list of numbers to data attribute
        else:
            raise Exception ('Only "Lists" are accepted as data.') # Raise an error if input is not a list

    def calc_sum(self):
        return sum(self.data) # Sum up all elements in the data list

    def calc_nbr_of_elements(self):
        return len(self.data) # Return the number of elements in the data list

    def calc_mean(self):
        return (self.calc_sum())/(self.calc_nbr_of_elements()) # Compute mean by dividing sum by the count of elements

    def print_summary(self):
        print ('Sum:', self.calc_sum())
        print('Number of elements:', self.calc_nbr_of_elements())
        print('Mean:', self.calc_mean())

In [16]:
L = [1, 2, 1, 3, 5, 7, 4, 9, 10, 3, 2, 1, 6, 4, 3, 2, 1, 10, 9, 1, 8, 7, 3, 2, 1]
my_data = DescriptiveStatistics() # Creates an instance of the class
my_data.add_data(L) # Adds list 'L' to the data attribute of 'my_data' instance using 'add_data' method

In [17]:
my_data.data

[1, 2, 1, 3, 5, 7, 4, 9, 10, 3, 2, 1, 6, 4, 3, 2, 1, 10, 9, 1, 8, 7, 3, 2, 1]

In [18]:
print('Sum:', my_data.calc_sum())
print('Number of elements:', my_data.calc_nbr_of_elements())
print('Mean:', my_data.calc_mean()) #Prints the results of the elements in list 'L' 

Sum: 105
Number of elements: 25
Mean: 4.2


In [19]:
my_data.print_summary()

Sum: 105
Number of elements: 25
Mean: 4.2


5. (b) If you change L to a tuple, an error is raised.

6.

In [20]:
class BankAccount:
    def __init__(self, account_holder, balance=0):
        self.account_holder= account_holder
        self.balance= balance

    def deposit(self, amount):
        self.balance += amount
        print(f'{amount} deposited. New balance {self.balance}')

    def withdraw(self, amount):
        if amount>self.balance:
            print("Balance too low.")
        else:
            self.balance -= amount
            print(f'{amount} withdrawn. New balance {self.balance}')

    

In [21]:
account= BankAccount("Shriya", 1000000)
account.deposit(1000000)
account.withdraw(50000)
account.withdraw(5000000)

1000000 deposited. New balance 2000000
50000 withdrawn. New balance 1950000
Balance too low.


7.

En klass är som en ritning för att skapa objekt. Den definierar en typ av objekt med specifika attribut (egenskaper) och metoder (åtgärder). Exempel: Bil

En instans är ett specifikt objekt skapat från en klass. När du skapar en instans skapar du ett objekt som följer strukturen som definieras av klassen. Exempel:En eller flera bilar

Attribut är variabler som hör till en instans. De har data relaterade till instansen. Exempel:Lagra information om varje bil: modell, år, färg, hastighet, osv.

Metoder är funktioner definierade inom en klass som beskriver de beteenden eller åtgärder som en instans kan utföra.Exempel: Definiera vad bilen kan göra, som att starta motorn 'start_engine', stoppa den 'stop engine' eller visa information om bilen 'car_info'.

### 7-Konventioner och standarder

1.(a) Jag skulle följa projektets stilriktlinjer istället för att strikt följa PEP 8-standarderna eftersom dessa riktlinjer inte är huggna i sten. Om företaget har valt att följa en viss stil, skulle jag bara vilja diskutera en avvikelse från den om jag anser att det är ineffektivt att fortsätta följa den. Min prioritet bör vara att samarbeta med teamet och anpassa mig till projektets specifika behov. 

 

(b) Engelska är det vanligaste språket för kommentarer. 

 

(c)Funktioner: namn i lägre fall med understreck mellan ord : ‘lowercase_with_underscores’. Namnet ska tydligt beskriva vad funktionen gör : ‘sum_weights’ 

Variabler: namn i lägre fall med understreck mellan ord : ‘lowercase_with_underscores’ Det ska beskriva vad variabeln respresenterar: ‘total_cost’. 

 

(d) Klassnamn bör använda CapWords, där varje ord börjar med en stor bokstav och inga understreck används, till exempel : ‘HouseGarden’  

 

(e) All import bör placeras högst upp i ditt skript. 

 

(f) PEP 8 anger inga specifika regler för användningen av enkla eller dubbla citattecken för strängar. Du kan använda båda stilarna så länge du är konsekvent i din kodbas. 

 

(g) Alternativ 1 är det korrekta sättet att skriva kod enligt PEP 8 eftersom det följer avståndsregeln. PEP 8 rekommenderar att du placerar ett enda utrymme runt binära operatorer, såsoma till förvirring och inkonsekvens. 

2.(a) I PEP 20 betyder "Explicit är bättre än implicit" att kod ska vara tydlig och okomplicerad. Undvik dolt eller tvetydigt beteende; gör din kods avsikt uppenbar och lätt att förstå. 

(b) I PEP 20 betyder "Simple is better than complex" att koden ska vara så enkel som möjligt. Sträva efter tydlighet och undvik onödiga komplikationer 

 
3.Att följa konventioner och standarder kan verka som ett krångel, men de förenklar faktiskt kodhantering och samarbete. Konsekvent kod är lättare att läsa, underhålla och felsöka, vilket i slutändan sparar tid och minskar fel. Om alla skriver kod som de vill kan det leda till förvirring och inkonsekvens. 

### 8-Fel hantering uppgifter

1. (a) Syntax fel är fel som uppstår när Python inte förstår koden. Till exempel: print("Hello, world!"
   Rätt kod: print("Hello, world!")

(b) Man skulle vilja fånga exceptions i ett program och inte bara låta programmet stanna vid fel för att det hjälper till att identifiera och logga       problem för senare felsökning utan att störa användarens upplevelse. Att fånga exceptions i ett program är viktigt eftersom det gör att programmet      kan hantera fel snarare än att krascha.

(c)Man skulle vilja lyfta exceptions i ett program för at ange när något går fel och gör det klart att programmets normala flöde inte kan fortsätta.       Det hjälper till att hitta exakt var och varför ett fel uppstod, vilket gör det lättare att identifiera och åtgärda problem.

2. (a)

In [22]:
def convert_string_to_int(string): #Denna funktion försöker konvertera en sträng till ett heltal.
    try:
        int(string)  # Attempt to convert the string to an integer
    except ValueError:
        return "Invalid input, cannot convert to integer."
        # If a ValueError occurs (e.g., the string is not a number),
        # return an error message
    else:
        # If no error occurs, return the converted integer
        return int(string)

# Test the function with a valid integer string
print(convert_string_to_int("315"))
# Test the function with an invalid integer string
print(convert_string_to_int("abc")) # Outputs: Invalid input, cannot convert to integer.

315
Invalid input, cannot convert to integer.


(b) Den skiljer koden som ska köras om allt går bra från koden som hanterar exceptions.

3.

In [23]:
def convert_to_upper(string):
    try:
        # Convert the string to uppercase using the correct method
        return string.upper()
    except AttributeError:
        # Catch an AttributeError if the input is not a string (e.g., if an integer is passed)
        return "Invalid input, cannot convert to upper."

print(convert_to_upper("hello, how are you?"))  # Outputs: HELLO, HOW ARE YOU?
print(convert_to_upper("562"))  # Outputs: 562

HELLO, HOW ARE YOU?
562


4.

In [24]:
def add_two_small_numbers(a,b):
    if a > 100 or b > 100:
        raise ValueError("Both numbers must be smaller than or equal to 100")
    return a + b

print(add_two_small_numbers(34, 71)) 
print(add_two_small_numbers(101, 50))

105


ValueError: Both numbers must be smaller than or equal to 100

5.

In [26]:
#Checking which exception is raised

try: 
    5 + "Python is fun!"
except Exception as exception_instance:
    print(type(exception_instance))
    print(exception_instance)

<class 'TypeError'>
unsupported operand type(s) for +: 'int' and 'str'


In [27]:
#Checking which exception is raised
try:
    5/0
except Exception as exception_instance:
     print(type(exception_instance))
     print(exception_instance)

<class 'ZeroDivisionError'>
division by zero


In [28]:
def add_two_numbers(a,b):
    try:
        return(a/b)
    except TypeError:
        print("Both arguments must be numbers.")
    except ZeroDivisionError:
        print("Division by zero is not defined.")

In [29]:
#Testing so the functionality is as expected
print(add_two_numbers(5,2))
print(add_two_numbers(5,"hello"))
print(add_two_numbers(5,0))

2.5
Both arguments must be numbers.
None
Division by zero is not defined.
None


Det här kodavsnittet visar hur man hanterar exceptions i Python genom att kontrollera vilka exceptions som uppstår under olika operationer och se till att en funktion beter sig korrekt när den stöter på fel.

### 9-Test uppgifter

1. Det är en god ide att bygga upp tester för sin kod när programmeringsprojekt växer i komplexitet. Man kan ha koll på olika delar av koden eller systemet med hjälp av tester. Tester hjälper ossatt säkerställa att befintlig kod fungerar som den ska.

2. doctest

In [30]:
def multiply_two_numbers(a,b):
    """Return the product of a and b. 
    >>> multiply_two_numbers(7,8)
    56
    """
    return a*b

import doctest
doctest.testmod()

TestResults(failed=0, attempted=1)

3.

In [32]:
def reverse_string(s):
    """
    Returns the reverse of the string s.

    >>> reverse_string("shriya")
    'ayirhs'
    >>> reverse_string("Python")
    'nohtyP'
    >>> reverse_string("theo")
    'theo'
    """
    return s[::-1]

import doctest
doctest.testmod()

**********************************************************************
File "__main__", line 9, in __main__.reverse_string
Failed example:
    reverse_string("theo")
Expected:
    'theo'
Got:
    'oeht'
**********************************************************************
1 items had failures:
   1 of   3 in __main__.reverse_string
***Test Failed*** 1 failures.


TestResults(failed=1, attempted=4)

### 10-Moduler och paket uppgifter

In [33]:
# Importerar modulen i en Jupyter Notebook
import my_module #importeras från vs code

# Test the addition function
result_add = my_module.addition(10, 5)
print(f"Addition of 10 and 5: {result_add}")

# Test the subtraction function
result_sub = my_module.subtraction(10, 5)
print(f"Subtraction of 10 and 5: {result_sub}")

Addition of 10 and 5: 15
Subtraction of 10 and 5: 5


### 11-Loggning uppgifter

1. Loggfiler med information kan vara viktiga för företag och organisationer för att de har flertalet processer som automatiskt kör programmeringskod via mölntjänster eller egna servrar så det inte är någon människa som manuellt kör koden. Då behöver ett sätt för att kolla hur körningarna har gått och vid eventuella fel eller problem förstå vad som har gått fel för att det ska kunna åtgärdas. Detta kan man göra genom att kolla på logg-filer där information från exekverad programmeringskod lagras.

2. Olika nivåer för loggmeddelanden innebär att loggar klassificeras efter deras allvarlighet eller betydelse. Vanliga nivåer är:

DEBUG: Detaljerad information för felsökning.  
INFO: Allmänna händelser som bekräftar att saker fungerar som förväntat.  
WARNING: Något oväntat hände, men programmet fortsätter att fungera.  
ERROR: Ett problem som förhindrade en funktion från att utföras.  
CRITICAL: Allvarligt fel som kan stoppa programmet.  

Dessa nivåer hjälper till att filtrera och prioritera logginformation.

In [34]:
import logging 

# Konfigurera logging 
logging.basicConfig(
    filename='bank.log',format='[%(asctime)s][%(levelname)s] %(message)s', level=logging.DEBUG, datefmt='%Y-%m-%d %H:%M')

class BankAccount:
    def __init__(self, account_holder, balance=0):
        self.account_holder= account_holder
        self.balance= balance
        logging.info(f'Created account for {self.account_holder} with balance {self.balance}')

    def deposit(self, amount):
        self.balance += amount
        print(f'{amount} deposited. New balance {self.balance}')

    def withdraw(self, amount):
        if amount>self.balance:
            logging.error(f'{self.account_holder} tried to withdraw {amount}, but balance is only {self.balance}')
            print("Balance too low.")
        elif amount <= 0:
            logging.warning(f'Tried to withdraw non-positive amount: {amount}')
        else:
            self.balance -= amount
            logging.info(f'{amount} withdrawn from {self.account_holder}\'s account. New balance: {self.balance}')
            
    def get_balance(self):
        logging.info(f'{self.account_holder} checked balance. Current balance: {self.balance}')
        
        return self.balance



In [35]:
# Testing the class
if __name__ == "__main__":
    account = BankAccount('Shriya', 100)
    account.deposit(50)
    account.withdraw(30)
    account.withdraw(150)  # This should log an error
    account.deposit(-10)    # Log a warning
    account.withdraw(0)     # Log a warning
    balance = account.get_balance()
    print(f'Shriya\'s final balance is: {balance}')

50 deposited. New balance 150
Balance too low.
-10 deposited. New balance 110
Shriya's final balance is: 110


In [36]:
!type bank.log

[2024-09-04 13:39][INFO] Created account for Shriya with balance 100
[2024-09-04 13:39][INFO] 30 withdrawn from Shriya's account. New balance: 120
[2024-09-04 13:39][ERROR] Shriya tried to withdraw 150, but balance is only 120
[2024-09-04 13:39][WARNING] Tried to withdraw non-positive amount: 0
[2024-09-04 13:39][INFO] Created account for Shriya with balance 100
[2024-09-04 13:39][INFO] 30 withdrawn from Shriya's account. New balance: 120
[2024-09-04 13:39][ERROR] Shriya tried to withdraw 150, but balance is only 120
[2024-09-04 13:39][WARNING] Tried to withdraw non-positive amount: 0
[2024-09-04 13:41][INFO] Created account for Shriya with balance 100
[2024-09-04 13:41][INFO] 30 withdrawn from Shriya's account. New balance: 120
[2024-09-04 13:41][ERROR] Shriya tried to withdraw 150, but balance is only 120
[2024-09-04 13:41][WARNING] Tried to withdraw non-positive amount: 0
[2024-09-04 13:44][INFO] Created account for Alice with balance 100
[2024-09-04 13:44][INFO] 30 withdrawn from A